Stefan Hannie

# Notebook for getting a nice table for graphing data

This notebook takes in the dataset and returns a table containing all unique pairs of people calling each other with relavent data necessary for creating a graph-like object visualized over a map of Canada. That is, it filters the cdr table for unique a_saddr and b_saddr pairs (unique rtp pairings). It then feeds this table through geolite (and ipaddress) to obtain human readable information such as ASN information and latitude/longitude.

It also creates smaller tables centered around Vancouver, Toronto, and Montreal (where CloudPBX has servers) in the instance that smaller graph-like objects and maps desire to be created.

In [10]:
import numpy as np
import pandas as pd
import geoip2.database
import ipaddress
import os
import dask.dataframe as dd

In [11]:
DATA_ROOT = 'files/'
CSV_FILE_PATH = os.path.join(DATA_ROOT + 'cdr_from_2018-05-01-order-calldate-asc.csv')
GEOLITE_ASN_PATH = os.path.join(DATA_ROOT + 'GeoLite2-ASN.mmdb')
GEOLITE_CITY_PATH = os.path.join(DATA_ROOT + 'GeoLite2-City.mmdb')

In [12]:
HEADER = ["ID","calldate","callend","duration","connect_duration","progress_time","first_rtp_time","caller",
          "caller_domain","caller_reverse","callername","callername_reverse","called","called_domain","called_reverse",
          "sipcallerip","sipcallerport","sipcalledip","sipcalledport","whohanged","bye","lastSIPresponse_id",
          "lastSIPresponseNum","sighup","dscp","a_index","b_index","a_payload","b_payload","a_saddr","b_saddr",
          "a_received","b_received","a_lost","b_lost","a_ua_id","b_ua_id","a_avgjitter_mult10","b_avgjitter_mult10",
          "a_maxjitter","b_maxjitter","a_sl1","a_sl2","a_sl3","a_sl4","a_sl5","a_sl6","a_sl7","a_sl8","a_sl9","a_sl10",
          "a_d50","a_d70","a_d90","a_d120","a_d150","a_d200","a_d300","b_sl1","b_sl2","b_sl3","b_sl4","b_sl5","b_sl6","b_sl7",
          "b_sl8","b_sl9","b_sl10","b_d50","b_d70","b_d90","b_d120","b_d150","b_d200","b_d300","a_mos_lqo_mult10",
          "b_mos_lqo_mult10","a_mos_f1_mult10","a_mos_f2_mult10","a_mos_adapt_mult10","b_mos_f1_mult10","b_mos_f2_mult10",
          "b_mos_adapt_mult10","a_rtcp_loss","a_rtcp_maxfr","a_rtcp_avgfr_mult10","a_rtcp_maxjitter","a_rtcp_avgjitter_mult10",
          "b_rtcp_loss","b_rtcp_maxfr","b_rtcp_avgfr_mult10","b_rtcp_maxjitter","b_rtcp_avgjitter_mult10","a_last_rtp_from_end",
          "b_last_rtp_from_end","payload","jitter_mult10","mos_min_mult10","a_mos_min_mult10","b_mos_min_mult10",
          "packet_loss_perc_mult1000","a_packet_loss_perc_mult1000","b_packet_loss_perc_mult1000","delay_sum","a_delay_sum",
          "b_delay_sum","delay_avg_mult100","a_delay_avg_mult100","b_delay_avg_mult100","delay_cnt","a_delay_cnt",
          "b_delay_cnt","rtcp_avgfr_mult10","rtcp_avgjitter_mult10","lost","id_sensor","price_operator_mult100",
          "price_operator_currency_id","price_customer_mult100","price_customer_currency_id","reason_sip_cause",
          "reason_sip_text_id","reason_q850_cause","reason_q850_text_id","caller_silence","called_silence",
          "caller_silence_end","called_silence_end","a_mos_xr_min_mult10","b_mos_xr_min_mult10","a_mos_xr_mult10",
          "b_mos_xr_mult10","response_time_100","response_time_xxx","a_mos_f1_min_mult10","a_mos_f2_min_mult10",
          "a_mos_adapt_min_mult10","b_mos_f1_min_mult10","b_mos_f2_min_mult10","b_mos_adapt_min_mult10","a_rtp_ptime",
          "b_rtp_ptime","flags"]

In [13]:
# read dataframe
df = pd.read_csv(CSV_FILE_PATH, names=HEADER)
df_sub = pd.read_csv(DATA_ROOT + 'distinct_subrip_cpbxip.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (95,97,98,99,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### make a table containing only rtp ip information

In [14]:
DESCRIBED_COLUMNS = ["a_saddr", "b_saddr"]
df_geo = df[DESCRIBED_COLUMNS]
df_geo.shape

(1374842, 2)

filtering out the carrier ips, keep only the service (?) ips

In [15]:
df_geo = df_geo.loc[df_geo['a_saddr'].isin(df_sub['subsriber_reg_ip']) | df_geo['b_saddr'].isin(df_sub['subsriber_reg_ip'])]
df_geo.shape

(601169, 2)

get rid of duplicate pairs

In [16]:
df_geo_unique_ip = df_geo.drop_duplicates()
df_geo_unique_ip.shape

(4072, 2)

count occurences of each pair in the original dataset

## Link IP addresses to ASN

In [17]:
# initiate geoip client
readerASN = geoip2.database.Reader(GEOLITE_ASN_PATH)
readerCITY = geoip2.database.Reader(GEOLITE_CITY_PATH)

In [18]:
# functions to get AS info
def getASobject(x):
    ip = ipaddress.ip_address(x)
    try: return readerASN.asn(str(ip))
    except: return "The address {} is not in the database.".format(ip)
def getIP(x):
    if type(x) == str: return x
    return x.ip_address
def getASN(x):
    if type(x) == str: return x
    return x.autonomous_system_number
def getASorg(x):
    if type(x) == str: return x
    return x.autonomous_system_organization
def getLat(x):
    try: return readerCITY.city(str(x)).location.latitude
    except: return 0.0
def getLong(x):
    try: return readerCITY.city(str(x)).location.longitude
    except: return 0.0

In [19]:
# making a vector of AS objects for sipcalled
V = df_geo_unique_ip['a_saddr'].apply(getASobject)
# adding columns to the data frame
df_geo_unique_ip['a_saddr_asn'] = V.apply(getASN)
df_geo_unique_ip['a_saddr_asorg'] = V.apply(getASorg)
df_geo_unique_ip['a_saddr_as_ip'] = V.apply(getIP)
df_geo_unique_ip['a_saddr_lat'] = df_geo_unique_ip['a_saddr_as_ip'].apply(getLat)
df_geo_unique_ip['a_saddr_long'] = df_geo_unique_ip['a_saddr_as_ip'].apply(getLong)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [20]:
# making a vector of AS objects for sipcalled
V = df_geo_unique_ip['b_saddr'].apply(getASobject)
# adding columns to the data frame
df_geo_unique_ip['b_saddr_asn'] = V.apply(getASN)
df_geo_unique_ip['b_saddr_asorg'] = V.apply(getASorg)
df_geo_unique_ip['b_saddr_as_ip'] = V.apply(getIP)
df_geo_unique_ip['b_saddr_lat'] = df_geo_unique_ip['b_saddr_as_ip'].apply(getLat)
df_geo_unique_ip['b_saddr_long'] = df_geo_unique_ip['b_saddr_as_ip'].apply(getLong)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

## df_geo_unique_ip holds the table but it has each individual ipaddress.
we don't want the ip addresses if they have the same lat/long

In [22]:
#df_geo_unique_ip.to_csv('unique_rtp_pairings_1M.csv')
df_geo_unique_ip.head()

,a_saddr,b_saddr,a_saddr_asn,a_saddr_asorg,a_saddr_as_ip,a_saddr_lat,a_saddr_long,b_saddr_asn,b_saddr_asorg,b_saddr_as_ip,b_saddr_lat,b_saddr_long
1,3227975250,1176778026,395152,CloudPBX,192.102.254.82,43.6319,-79.3716,40029,Novus Entertainment Inc.,70.36.53.42,49.2788,-123.1139
2,1614226523,3227975250,6327,Shaw Communications Inc.,96.55.36.91,49.1889,-122.8730,395152,CloudPBX,192.102.254.82,43.6319,-79.3716
9,3227975251,1268628388,395152,CloudPBX,192.102.254.83,43.6319,-79.3716,852,TELUS Communications Inc.,75.157.187.164,49.8928,-119.4852
10,1268497742,3227975250,852,TELUS Communications Inc.,75.155.189.78,49.2475,-123.1210,395152,CloudPBX,192.102.254.82,43.6319,-79.3716
14,3227975250,3330677851,395152,CloudPBX,192.102.254.82,43.6319,-79.3716,30500,Wolfpaw Data Centres Inc,198.134.28.91,55.3333,-115.4167


In [23]:
df_geo_unique_ip.shape

(4072, 12)

## let's filter out ips with the same lat/long
df_geo_unique_asn holds the table filtered for unique asn

In [24]:
df_geo_unique_latlong = df_geo_unique_ip[['a_saddr_asn','a_saddr_asorg','a_saddr_lat','a_saddr_long',
                                   'b_saddr_asn','b_saddr_asorg','b_saddr_lat','b_saddr_long']].drop_duplicates()

In [25]:
df_geo_unique_latlong.head()

,a_saddr_asn,a_saddr_asorg,a_saddr_lat,a_saddr_long,b_saddr_asn,b_saddr_asorg,b_saddr_lat,b_saddr_long
1,395152,CloudPBX,43.6319,-79.3716,40029,Novus Entertainment Inc.,49.2788,-123.1139
2,6327,Shaw Communications Inc.,49.1889,-122.8730,395152,CloudPBX,43.6319,-79.3716
9,395152,CloudPBX,43.6319,-79.3716,852,TELUS Communications Inc.,49.8928,-119.4852
10,852,TELUS Communications Inc.,49.2475,-123.1210,395152,CloudPBX,43.6319,-79.3716
14,395152,CloudPBX,43.6319,-79.3716,30500,Wolfpaw Data Centres Inc,55.3333,-115.4167


In [26]:
df_geo_unique_latlong.shape

(1440, 8)

## Split into cities

In [28]:
df_van_ip = df_geo_unique_ip[df_geo_unique_ip['a_saddr_asn']==395152]
df_van_latlong = df_geo_unique_latlong[df_geo_unique_latlong['a_saddr_asn']==395152]
df_tor_ip = df_geo_unique_ip[df_geo_unique_ip['a_saddr_asn']==393755]
df_tor_latlong = df_geo_unique_latlong[df_geo_unique_latlong['a_saddr_asn']==393755]
df_mtl_ip = df_geo_unique_ip[df_geo_unique_ip['a_saddr_asn']==395766]
df_mtl_latlong = df_geo_unique_latlong[df_geo_unique_latlong['a_saddr_asn']==395766]

In [29]:
print(df_van_ip.shape,
df_tor_ip.shape,
df_mtl_ip.shape)

(676, 12) (765, 12) (532, 12)


In [30]:
print(df_van_latlong.shape,
df_tor_latlong.shape,
df_mtl_latlong.shape)

(251, 8) (249, 8) (205, 8)


In [32]:
df_van_latlong.to_csv('van_rtp_unique_latlong.csv')
df_tor_latlong.to_csv('tor_rtp_unique_latlong.csv')
df_mtl_latlong.to_csv('mtl_rtp_unique_latlong.csv')